In [1]:
import pandas
import pandapower


net = pandapower.converter.from_mpc('../estimator_data/WP 4/A_KPC_35/A_KPC_35.mat')

data_root = '../estimator_data/WP 4/A_KPC_35/Loading/KPC_35_'

def get_data(season, day, bus):
    bus_network = bus
    bus = str(bus + 1)
    
    result = pandas.DataFrame()
    read = pandas.read_csv(data_root + f'{season}_{day}.csv', header=[0, 1])
    result[('goal', 'p')] = read[bus]['P']
    result[('goal', 'q')] = read[bus]['Q']

    seasons = ['spring', 'summer', 'autumn', 'winter']
    season = {'S': 'spring', 'Su': 'summer', 'A': 'autumn', 'W': 'winter'}[season]
    for s in seasons:
        result[('season', s)] = False if s != season else True
    
    day_types = ['business', 'saturday', 'sunday']
    day = {'Bd': 'business', 'Sa': 'saturday', 'Su': 'sunday'}[day]
    for dt in day_types:
        result[('day', dt)] = False if dt != day else True

    result[('time', 'hour')] = [i // 4 for i in range(len(result))]
    result[('time', 'minute')] = [i % 4 * 15 for i in range(len(result))]
    result[('bus', 'p')] = (net.load[net.load.bus == bus_network].p_mw.sum()
                            - net.sgen[net.sgen.bus == bus_network].p_mw.sum())
    result[('bus', 'q')] = (net.load[net.load.bus == bus_network].q_mvar.sum()
                            - net.sgen[net.sgen.bus == bus_network].q_mvar.sum())
    result[('bus', 'voltage_level')] = net.bus.iloc[bus_network].vn_kv
    result.columns = pandas.MultiIndex.from_tuples(result.columns)

    return result


def get_single_bus(bus):
    result = pandas.DataFrame()
    for season in ('S', 'Su', 'A', 'W'):
        for day in ('Bd', 'Sa', 'Su'):
            subset = get_data(season, day, bus)
            result = pandas.concat((result, subset), ignore_index=True)
    return result


def get_total():
    total = pandas.DataFrame()
    for bus in range(0, 14):
        bus_frame = get_single_bus(bus)
        total = pandas.concat((total, bus_frame), ignore_index=True)
    return total

In [2]:
total = get_total()

total

goal           season                           day           \
              p         q spring summer autumn winter business saturday   
0      4.673515  0.538333   True  False  False  False     True    False   
1      4.664708  0.571190   True  False  False  False     True    False   
2      4.630194  0.549504   True  False  False  False     True    False   
3      4.660333  0.528940   True  False  False  False     True    False   
4      4.748231  0.474548   True  False  False  False     True    False   
...         ...       ...    ...    ...    ...    ...      ...      ...   
16165 -0.311830  0.208304  False  False  False   True    False    False   
16166 -0.299094  0.213987  False  False  False   True    False    False   
16167 -0.278665  0.215356  False  False  False   True    False    False   
16168 -0.266018  0.212297  False  False  False   True    False    False   
16169 -0.262974  0.210127  False  False  False   True    False    False   

             time         bus                      
      sunday hour minute    p     q voltage_level  
0      False    0      0  0.0  0.00         110.0  
1      False    0     15  0.0  0.00         110.0  
2      False    0     30  0.0  0.00         110.0  
3      False    0     45  0.0  0.00         110.0  
4      False    1      0  0.0  0.00         110.0  
...      ...  ...    ...  ...   ...           ...  
16165   True   22     45  1.2  0.02          10.0  
16166   True   23      0  1.2  0.02          10.0  
16167   True   23     15  1.2  0.02          10.0  
16168   True   23     30  1.2  0.02          10.0  
16169   True   23     45  1.2  0.02          10.0  

[16170 rows x 14 columns]

In [3]:
total.to_csv('../datasets/kpc.csv', index=False)

In [34]:
index = 0

measurements = pandas.DataFrame()

for bus_id in range(14):
    bus_measurements = get_single_bus(bus_id)
    item = bus_measurements.iloc[index]
    measurements = pandas.concat(
        [measurements, pandas.DataFrame(
            {'meas_type': 'p', 'element_type': 'bus',
             'value': item.goal.p, 'std_dev': 0.1, 'element': bus_id}, index=[0])],
        ignore_index=True)
    measurements = pandas.concat(
        [measurements, pandas.DataFrame(
            {'meas_type': 'q', 'element_type': 'bus',
             'value': item.goal.q, 'std_dev': 0.1, 'element': bus_id}, index=[0])],
        ignore_index=True)
    
measurements

,meas_type,element_type,value,std_dev,element
0,p,bus,4.673515,0.1,0
1,q,bus,0.538333,0.1,0
2,p,bus,1.469946,0.1,1
3,q,bus,-0.128578,0.1,1
4,p,bus,3.554882,0.1,2
5,q,bus,-0.498497,0.1,2
6,p,bus,0.414527,0.1,3
7,q,bus,-0.593766,0.1,3
8,p,bus,3.203330,0.1,4
9,q,bus,-0.654012,0.1,4


In [33]:
measurements.to_csv('../datasets/measurements.csv', index=None)

In [30]:
total

goal           season                           day           \
              p         q spring summer autumn winter business saturday   
0      4.673515  0.538333   True  False  False  False     True    False   
1      4.664708  0.571190   True  False  False  False     True    False   
2      4.630194  0.549504   True  False  False  False     True    False   
3      4.660333  0.528940   True  False  False  False     True    False   
4      4.748231  0.474548   True  False  False  False     True    False   
...         ...       ...    ...    ...    ...    ...      ...      ...   
16165 -0.311830  0.208304  False  False  False   True    False    False   
16166 -0.299094  0.213987  False  False  False   True    False    False   
16167 -0.278665  0.215356  False  False  False   True    False    False   
16168 -0.266018  0.212297  False  False  False   True    False    False   
16169 -0.262974  0.210127  False  False  False   True    False    False   

             time         bus                      
      sunday hour minute    p     q voltage_level  
0      False    0      0  0.0  0.00         110.0  
1      False    0     15  0.0  0.00         110.0  
2      False    0     30  0.0  0.00         110.0  
3      False    0     45  0.0  0.00         110.0  
4      False    1      0  0.0  0.00         110.0  
...      ...  ...    ...  ...   ...           ...  
16165   True   22     45  1.2  0.02          10.0  
16166   True   23      0  1.2  0.02          10.0  
16167   True   23     15  1.2  0.02          10.0  
16168   True   23     30  1.2  0.02          10.0  
16169   True   23     45  1.2  0.02          10.0  

[16170 rows x 14 columns]